In [1]:
import shodan
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
from string import Template
import json
import networkx as nx
import re

In [2]:
nodes = set()
edges = set()

for banner in shodan.helpers.iterate_files('isp1k.json.gz'):
    nodes.add(banner['isp'])
    nodes.add(banner['ip_str'])
    edges.add((banner['ip_str'], banner['isp']))

In [3]:
G = nx.Graph()
G.add_nodes_from(nodes)
G.add_edges_from(edges)
pos = nx.nx_agraph.graphviz_layout(G)

In [4]:
graph = {'nodes': [], 'edges': []}
for node in nodes:
    graph['nodes'].append({
        'id': node,
        'label': node,
        'x': pos[node][0],
        'y': pos[node][1],
        'color': 'black' if re.match(r'^\d+\.', node) else 'red'
    })

for i, edge in enumerate(edges):
    graph['edges'].append({
        'id': i,
        'source': edge[0],
        'target': edge[1],
        'color': 'rgba(200, 200, 200, .5)'
    })

print(len(graph['nodes']))
print(graph)

995
{'nodes': [{'id': '217.165.24.214', 'label': '217.165.24.214', 'x': 954.02, 'y': 985.68, 'color': 'black'}, {'id': '176.205.86.154', 'label': '176.205.86.154', 'x': 1074.6, 'y': 882.0, 'color': 'black'}, {'id': '176.205.23.8', 'label': '176.205.23.8', 'x': 1014.6, 'y': 1095.2, 'color': 'black'}, {'id': '151.253.98.34', 'label': '151.253.98.34', 'x': 1126.5, 'y': 958.92, 'color': 'black'}, {'id': '194.170.169.158', 'label': '194.170.169.158', 'x': 1030.6, 'y': 1012.6, 'color': 'black'}, {'id': '94.200.119.12', 'label': '94.200.119.12', 'x': 619.79, 'y': 730.82, 'color': 'black'}, {'id': '94.200.162.158', 'label': '94.200.162.158', 'x': 737.8, 'y': 642.41, 'color': 'black'}, {'id': '94.59.249.181', 'label': '94.59.249.181', 'x': 1001.4, 'y': 908.07, 'color': 'black'}, {'id': '86.98.31.220', 'label': '86.98.31.220', 'x': 1156.2, 'y': 929.93, 'color': 'black'}, {'id': '91.74.32.2', 'label': '91.74.32.2', 'x': 672.41, 'y': 849.8, 'color': 'black'}, {'id': '94.56.177.26', 'label': '94.56

In [5]:
js_template = Template('''
graph = $graph_data

S = new sigma({
  graph: graph,
  renderer: {
    container: 'container',
    type: 'canvas'
  }
})

sigma.plugins.relativeSize(S, 1)

S.startNoverlap()

S.refresh()
''')

html_template = Template('''
<title>IPISP</title>
<meta charset='utf-8'>
<h1 style='font-family:sans-serif;text-align:center'>$title</h1>
<p style='font-family:sans-serif;text-align:center'>Drag, click, and scroll on the map to interact.</p>
<br>
<div id='container' style='height:800px'></div>
<script src='https://cdnjs.cloudflare.com/ajax/libs/sigma.js/1.2.1/sigma.min.js'></script>
<script src='https://cdnjs.cloudflare.com/ajax/libs/sigma.js/1.2.1/plugins/sigma.plugins.relativeSize.min.js'></script>
<script src='https://cdnjs.cloudflare.com/ajax/libs/sigma.js/1.2.1/plugins/sigma.layout.noverlap.min.js'></script>
<script>$js_text</script>
''')

In [6]:
js_text = js_template.substitute({
    'graph_data': json.dumps(graph)
})

h = html_template.substitute({
    'title': '1000 Exposed IPs in the UAE',
    'js_text': js_text
})

f = open('index.html', 'w+')
f.write(h)
f.close()

HTML(h)